In [49]:
import pandas as pd
import numpy as np
from functools import cache
from IPython.display import display


pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns


sourceDf = pd.read_csv("./data/Europe-Central-Asia_2018-2024_Sep27.csv")
print(sourceDf.columns)

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp'],
      dtype='object')


In [50]:
unique_sub_event = sourceDf['sub_event_type'].unique()
unique_sub_event = np.sort(unique_sub_event)

non_war_related_event  = [
    'Agreement',
    'Arrests',
    'Mob violence',
    'Excessive force against protesters',
    'Peaceful protest',
    'Protest with intervention',
    'Sexual violence',
    'Violent demonstration',
    'Looting/property destruction'
]

war_related_event =  [event for event in unique_sub_event if event not in non_war_related_event ]


ukraine_russia_events = sourceDf[(sourceDf["country"] == "Ukraine") | (sourceDf["country"] == "Russia")]
ukraine_war_events = ukraine_russia_events[ukraine_russia_events["sub_event_type"].isin(war_related_event)]

id_date_note_triplet = ukraine_war_events[["event_date", "notes", "event_id_cnty"]]
id_date_note_triplet = id_date_note_triplet.sort_values(by = ["event_date", "notes", "event_id_cnty"])


display(id_date_note_triplet.head(5))


,event_date,notes,event_id_cnty
424080,2018-01-01,"On 1 January 2018, Military Forces of Ukraine fired at the outskirts of Donetsk (Volvo Centre). The OSCE SMM observed a total of 9 bursts of fire involving small arms near Donetsk.",UKR18
424125,2018-01-01,"On 1 January 2018, Military Forces of Ukraine fired with 82 mm mortars at NAF near Yasynuvata. This day one DPR soldier was injured in Donbass region in the result of the shelling.",UKR32
424124,2018-01-01,"On 1 January 2018, Military Forces of Ukraine fired with 82 mm mortars at NAF rebel forces near Sosnovskoye. This day one DPR soldier was injured in Donbass region in the result of the shelling.",UKR29
424062,2018-01-01,"On 1 January 2018, Military Forces of Ukraine fired with recoilless guns, automatic grenade launchers and small arms at LPR Luhansk People's Militia positions (NAF) near Kalynivka.",UKR22
424074,2018-01-01,"On 1 January 2018, Military Forces of Ukraine fired with unspecified artillery on Zaytseve in Donetsk region. No casualties were reported.",UKR33


In [51]:
max_edit_distance = 5

def is_similar(rep_sentence, cur_sentence):

    rep_words = rep_sentence.split(" ")
    cur_words = cur_sentence.split(" ")

    @cache
    def compute_edit_distance(i, j):
        if i >= len(rep_words):
            return len(cur_words) - j
        
        if j >= len(cur_words):
            return len(rep_words) - j

        if rep_words[i] == cur_words[j]:
            return compute_edit_distance(i+1, j+1)
        
        return 1 + min(
            compute_edit_distance(i+1, j+1),
            compute_edit_distance(i, j+1),
            compute_edit_distance(i+1, j)
        )


    # seems that related notes all have a comment of "coded to xxx"
    keyword = "coded to"

    if keyword not in rep_sentence or (keyword not in cur_sentence):
        return False
    if rep_words[:5] != cur_words[:5]:
        return False

    dist =  compute_edit_distance(0,0) 

    return dist < max_edit_distance


s1 = "On 4 October 2022, following clashes with Russian forces, Ukrainian forces regained control over Biliaivka, Kherson region. 80 Ukrainian soldiers were killed across 17 locations. Casualties on Russian side unknown. [Russian MoD reported 80 fatalities. Coded as 10 fatalities split across 17 events. 1 fatality coded to this event]"
s2 = "On 4 October 2022, following clashes with Russian forces, Ukrainian forces regained control over Chereshneve, Kherson region. 80 Ukrainian soldiers were killed across 17 locations. Casualties on Russian side unknown. [Russian MoD reported 80 fatalities. Coded as 10 fatalities split across 17 events. 1 fatality coded to this event]"

print(is_similar(s1, s2))

        

True


In [52]:




# compute a mapping of event_id_cnty -> event_id_cnty where similar event should be mapped to the same representative event_id_cnty
# we already sorted the records so similiar notes should be adjacent

mapping = dict()

prev_event_id = None
prev_note = "XXX XXX"
prev_date = None


for row in id_date_note_triplet.itertuples():
    cur_event_id = row.event_id_cnty
    cur_date = row.event_date
    cur_note = row.notes


    if cur_date == prev_date and is_similar(prev_note, cur_note):
        mapping[cur_event_id] = prev_event_id
    else:
        prev_date = cur_date
        prev_note = cur_note
        prev_event_id = cur_event_id
        mapping[cur_event_id] = cur_event_id

# print({x:y for x, y in mapping.items() if x != y})


In [53]:
# targets = "|".join(["UKR47730", "UKR47716"])
# ukraine_war_events = ukraine_war_events[ukraine_war_events["event_id_cnty"].str.contains(targets)]
# display(ukraine_war_events[ukraine_war_events["event_id_cnty"].str.contains(targets)])


# incorporate this mapping as a new key in the dataframe


def group_key(x):
    return mapping[x]


projected = ukraine_war_events[["event_id_cnty", "event_date", "sub_event_type", "actor1", "actor2", "latitude", "notes", "longitude", "fatalities"]]
aggregated = projected.groupby(projected["event_id_cnty"].apply(group_key)).agg({
    "event_id_cnty": "first",
    "event_date": "first",
    "sub_event_type": "first",
    "actor1": "first",
    "actor2": "first",
    "notes": "first",
    "latitude": lambda l: ','.join([str(x) for x in l]),
    "longitude": lambda l: ','.join([str(x) for x in l]),
    "fatalities": 'sum'
})

display(aggregated.head(10))





,event_id_cnty,event_date,sub_event_type,actor1,actor2,notes,latitude,longitude,fatalities
event_id_cnty,,,,,,,,,
RUS1,RUS1,2018-01-03,Armed clash,Unidentified Armed Group (Russia),Police Forces of Russia (2000-),"On 3 January 2018, Abubakar Ustarkhanov, the local police chief from Avtury village (Shalinskiy District, Chechnya) was shot dead, as he was leaving a local shop and the murderer allegedly escaped in a police car.",43.0948,46.0043,1
RUS10,RUS10,2018-01-08,Grenade,Unidentified Armed Group (Russia),None,"On 8 January 2018, a grenade was thrown in Nazran on a house of the family of Magomed Khazbiyev, a member of opposition in Ingushetia, destroying a car.",43.2152,44.7761,0
RUS1000,RUS1000,2018-08-03,Remote explosive/landmine/IED,Unidentified Armed Group (Russia),None,"On 3 August 2018, an explosive device damaged the entrance to a national Pension Fund office in Kaluga city, Kaluga Municipality, Kaluga Oblast. No information on attackers, no casualties.",54.5293,36.2754,0
RUS10000,RUS10000,2023-05-29,Disrupted weapons use,Military Forces of Russia (2000-),Military Forces of Ukraine (2019-) Air Force,"Interception: On 29 May 2023, Russian air defense shot down a (presumably) Ukrainian drone near Staroselye.",50.705,35.4653,0
RUS10001,RUS10001,2023-05-30,Disrupted weapons use,Military Forces of Russia (2000-),Military Forces of Ukraine (2019-) Air Force,"Interception: On 30 May 2023, Russian air defense shot down a (presumably) Ukrainian drone over Antonovka.",50.5501,35.7583,0
RUS10002,RUS10002,2023-05-30,Disrupted weapons use,Military Forces of Russia (2000-),Military Forces of Ukraine (2019-) Air Force,"Interception: On 30 May 2023, Russian air defense shot down several (presumably) Ukrainian drones over Klimovskiy district. (coded to Klimovo)",52.3833,32.1833,0
RUS10003,RUS10003,2023-05-27,Disrupted weapons use,Military Forces of Russia (2000-),Unidentified Armed Group (Russia),"Interception: On 27 May 2023, Russian air defense shot down several unidentified drones over Ilskiy, close to the Ilskiy oil refinery plant.",44.8423,38.5681,0
RUS10004,RUS10004,2023-05-30,Disrupted weapons use,Military Forces of Russia (2000-),Unidentified Armed Group (Russia),"Interception: On 30 May 2023, Russian air defense shot down at least 8 drones over Moscow and Moscow region, with some sources claiming that overall more than 30 drones were intercepted. Several drones were shot down over Ilyinskoe, Moscow region, in the area where Russia's elite have luxurious property. While Moscow blamed Ukraine for the attack, Ukraine denied responsibility. According to some analysts, one of the drones captured on video resembled a Ukrainian UJ-22 drone.",55.757,37.242,0
RUS10005,RUS10005,2023-05-30,Disrupted weapons use,Military Forces of Russia (2000-),Unidentified Armed Group (Russia),"Interception: On 30 May 2023, Russian air defense shot down at least 8 drones over Moscow and Moscow region, with some sources claiming that overall more than 30 drones were intercepted. One drone was shot down over Moscow - Novomoskovsky, Moscow region. While Moscow blamed Ukraine for the attack, Ukraine denied responsibility. According to some analysts, one of the drones captured on video resembled a Ukrainian UJ-22 drone.",55.5987,37.3517,0
